In [ ]:
import numpy as np
import pandas as pd
import os
from scipy import stats
import utils
import torch.optim as optim
import torch
from torch.utils.data.sampler import RandomSampler

import model.net as net
from dataloader import *
from train import train_and_evaluate
from evaluate import evaluate

months = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

data_set = 'London_2013'
path = os.path.abspath(os.path.join(os.getcwd(), '../..'))

data = get_data(path, data_set)

In [ ]:
def qloss(y_true, y_pred):
    q = np.array(range(1, 100))
    tmp1 = (q / 100 - 1) * (y_true - y_pred)
    tmp2 = q / 100 * (y_true - y_pred)
    return np.mean(np.maximum(tmp1, tmp2))

def ws(alpha, l, u, y_true):
    total_ws = 0
    for i in range(len(y_true)):
        if y_true[i] >= l[i] and y_true[i] <= u[i]:
            ws = u[i] - l[i]
            total_ws = total_ws + ws
        if y_true[i] < l[i]:
            ws = u[i] - l[i] + 2 * (l[i] - y_true[i]) / alpha
            total_ws = total_ws + ws
        if y_true[i] > u[i]:
            ws = u[i] - l[i] + 2 * (y_true[i] - u[i]) / alpha
            total_ws = total_ws + ws
    return total_ws/len(y_true)

In [ ]:
total_times = 1
error_qs = np.zeros((total_times, 12, 10))
error_ws_90 = np.zeros((total_times, 12, 10))
error_ws_50 = np.zeros((total_times, 12, 10))

method = 'kmeans'

for times in trange(1, total_times+1):
    for month in trange(1, 13):
        for n_clusters in range(1, 11):

            path_cluster = os.path.join(path, 'result', data_set, 'clustering', 'point', method, f'n_clusters_{n_clusters}.csv')
            clusters = pd.read_csv(path_cluster, header=None)

            series = data[:, month-1, :months[month-1]*24].T.copy()

            window_size = 192
            stride_size = 24

            total_time = series.shape[0]
            num_series = series.shape[1]

            weather = get_weather(path, data_set, month)
            week = get_dow(data_set, month)
            day = get_hod(month)

            num_covariates = 4
            covariates = np.zeros((num_covariates, len(series)))
            covariates[1] = stats.zscore(weather)
            covariates[2] = stats.zscore(week)
            covariates[3] = stats.zscore(day)
            cov_age = stats.zscore(np.arange(total_time))
            covariates[0] = cov_age
            covariates = covariates.T.copy()

            # params
            json_path = os.path.join(path, 'forecasting', 'deepar', 'params24.json')
            params = utils.Params(json_path)

            params.num_class = n_clusters
            params.relative_metrics = False
            params.sampling = False
            params.one_step = True

            model_dir = os.path.join(path, 'result', data_set, 'forecasting', 'deepar', f'times_{times}', method)
            if not os.path.exists(model_dir):
                os.makedirs(model_dir)
            params.model_dir = os.path.join(model_dir, f'n_clusters_{n_clusters}_month_{month}.pth.tar')

            # use GPU if available
            cuda_exist = torch.cuda.is_available()

            # Set random seeds for reproducible experiments if necessary
            if cuda_exist:
                params.device = torch.device('cuda')
                # torch.cuda.manual_seed(240)
                model = net.Net(params).cuda()
            else:
                params.device = torch.device('cpu')
                # torch.manual_seed(230)
                model = net.Net(params)

            optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)
            loss_fn = net.loss_fn

            restore_file = None

            model = net.Net(params)
            utils.load_checkpoint(params.model_dir, model)

            total_mu = []
            total_sigma = []

            for day in range(1, 8):
                if day != 7:
                    test_data = series[-168-(8-day)*24:-(8-day)*24+24, :].copy()
                    cov = covariates[-168-(8-day)*24:-(8-day)*24+24, :].copy()
                else:
                    test_data = series[-192:, :].copy()
                    cov = covariates[-192:, :].copy()

                test_x_input, test_v_input, test_label = prep_data(test_data, cov, window_size, stride_size, num_covariates, num_series, clusters[month-1], train=False)
                test_set = TestDataset(test_x_input, test_v_input, test_label)
                test_loader = DataLoader(test_set, batch_size=params.predict_batch, sampler=RandomSampler(test_set), num_workers=4)

                test_metrics, mu, sigma = evaluate(model, loss_fn, test_loader, params, sample=params.sampling)

                day_mu = torch.zeros(24)
                day_sigma = torch.zeros(24)
                for i in range(len(mu)):
                    day_mu = day_mu + torch.sum(mu[i], axis=0)
                    day_sigma = day_sigma + torch.sum(sigma[i]**2, axis=0)
                total_mu.append(day_mu.numpy())
                total_sigma.append(torch.sqrt(day_sigma).numpy())

            total_mu = np.array(total_mu).reshape(-1)
            total_sigma = np.array(total_sigma).reshape(-1)

            pred = np.zeros((168, 99))

            for i in range(len(total_mu)):
                tmp = np.random.normal(total_mu[i], total_sigma[i], 10000)
                for j in range(99):
                    pred[i][j] = np.percentile(tmp, j+1)

            test = np.sum(series, axis=1)[-168:]

            error_qs[times-1, month-1, n_clusters-1] = qloss(np.tile(test, (99,1)).T, pred)
            error_ws_90[times-1, month-1, n_clusters-1] = ws(0.1, pred[:, 4], pred[:, 94], test)
            error_ws_50[times-1, month-1, n_clusters-1] = ws(0.5, pred[:, 24], pred[:, 74], test)

In [ ]:
pd.DataFrame(np.median(error_ws_90, axis=0)[7, :]).to_csv('tmp.csv')

In [ ]:
for month in trange(1, 13):
    for n_clusters in range(1, 2):

        path_cluster = os.path.join(path, 'result', data_set, 'clustering', 'point', 'kmeans', f'n_clusters_{n_clusters}.csv')
        clusters = pd.read_csv(path_cluster, header=None)

        series = data[:, month-1, :months[month-1]*24].T.copy()

        window_size = 192
        stride_size = 24

        total_time = series.shape[0]
        num_series = series.shape[1]

        weather = get_weather(path, data_set, month)
        week = get_dow(data_set, month)
        day = get_hod(month)

        num_covariates = 4
        covariates = np.zeros((num_covariates, len(series)))
        covariates[1] = stats.zscore(weather)
        covariates[2] = stats.zscore(week)
        covariates[3] = stats.zscore(day)
        cov_age = stats.zscore(np.arange(total_time))
        covariates[0] = cov_age
        covariates = covariates.T.copy()

        # params
        json_path = os.path.join(path, 'forecasting', 'deepar', 'params24.json')
        params = utils.Params(json_path)

        params.num_class = n_clusters
        params.relative_metrics = False
        params.sampling = False
        params.one_step = True

        model_dir = os.path.join(path, 'result', data_set, 'forecasting', 'deepar', f'times_{1}', method)
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        params.model_dir = os.path.join(model_dir, f'n_clusters_{n_clusters}_month_{month}.pth.tar')

        # use GPU if available
        cuda_exist = torch.cuda.is_available()

        # Set random seeds for reproducible experiments if necessary
        if cuda_exist:
            params.device = torch.device('cuda')
            # torch.cuda.manual_seed(240)
            model = net.Net(params).cuda()
        else:
            params.device = torch.device('cpu')
            # torch.manual_seed(230)
            model = net.Net(params)

        optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)
        loss_fn = net.loss_fn

        restore_file = None

        model = net.Net(params)
        utils.load_checkpoint(params.model_dir, model)

        total_mu = []
        total_sigma = []

        for day in range(1, 8):
            if day != 7:
                test_data = series[-168-(8-day)*24:-(8-day)*24+24, :].copy()
                cov = covariates[-168-(8-day)*24:-(8-day)*24+24, :].copy()
            else:
                test_data = series[-192:, :].copy()
                cov = covariates[-192:, :].copy()

            test_x_input, test_v_input, test_label = prep_data(test_data, cov, window_size, stride_size, num_covariates, num_series, clusters[month-1], train=False)
            test_set = TestDataset(test_x_input, test_v_input, test_label)
            test_loader = DataLoader(test_set, batch_size=params.predict_batch, sampler=RandomSampler(test_set), num_workers=4)

            test_metrics, mu, sigma = evaluate(model, loss_fn, test_loader, params, sample=params.sampling)

            day_mu = torch.zeros(24)
            day_sigma = torch.zeros(24)
            for i in range(len(mu)):
                day_mu = day_mu + torch.sum(mu[i], axis=0)
                day_sigma = day_sigma + torch.sum(sigma[i]**2, axis=0)
            total_mu.append(day_mu.numpy())
            total_sigma.append(torch.sqrt(day_sigma).numpy())

        total_mu = np.array(total_mu).reshape(-1)
        total_sigma = np.array(total_sigma).reshape(-1)

        pred = np.zeros((168, 99))

        for i in range(len(total_mu)):
            tmp = np.random.normal(total_mu[i], total_sigma[i], 10000)
            for j in range(99):
                pred[i][j] = np.percentile(tmp, j+1)

        test = np.sum(series, axis=1)[-168:]

        error_qs[0, month-1, 0] = qloss(np.tile(test, (99,1)).T, pred)
        error_ws_90[0, month-1, 0] = ws(0.1, pred[:, 4], pred[:, 94], test)
        error_ws_50[0, month-1, 0] = ws(0.5, pred[:, 24], pred[:, 74], test)

In [ ]:
error_qs[0, :, 0]

In [ ]:
error_ws_50[0, :, 0]

In [ ]:
error_ws_90[0, :, 0]